# Install VideoImageTools

To install VideoImageTools go to next link https://github.com/trucomanx/VideoImageTools

# Install OpenPifPafTools

To install OpenPifPafTools go to next link https://github.com/trucomanx/OpenPifPafTools


In [ ]:
input_default_json_conf_file='cnn_emotion4_testing_over_video_default.json';

# Bibliotecas externas

In [6]:
import platform, sys, os

import VideoImageTools as vit
import OpenPifPafTools.OpenPifPafAnnotations as opp
import OpenPifPafTools.OpenPifPafGetData as oppgd

import cv2
import openpifpaf
from PIL import Image

In [ ]:
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Biblioteca local

In [4]:
sys.path.append('../library');
import BodyEmotion4Lib.Classifier as mylib

# Variables

In [ ]:
## Load json conf file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

In [2]:
## Model of network
## 'mobilenet_v3', 'efficientnet_b3', 'inception_v3', 'inception_resnet_v2', 'resnet_v2_50'
model_type = DATA['model_type'];

dataset_name = DATA['dataset_name'];

vin_path = DATA['input_mp4_file'];

output_base_dir = DATA['output_base_dir'];

file_of_weight = DATA['model_weights_file'];;

# If command line

In [7]:
for n in range(len(sys.argv)):
    if sys.argv[n]=='--model':
        model_type=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--input-file':
        vin_path=sys.argv[n+1];
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--weights-file':
        file_of_weight=sys.argv[n+1];

print('     model_type:',model_type);
print('   dataset_name:',dataset_name);
print('       vin_path:',vin_path);
print('output_base_dir:',output_base_dir);
print(' file_of_weight:',file_of_weight);

model_type: efficientnet_b3


In [ ]:
vout_dir_path=os.path.join(output_base_dir,dataset_name,'test_over_video')

In [8]:

vout_path=os.path.join(vout_dir_path,os.path.splitext(os.path.basename(input_mp4_file))[0]+'_'+model_type+'.mp4');


# Creating output directory

In [9]:

os.makedirs(vout_dir_path,exist_ok=True); 
print('Created output_dir::',vout_dir_path);

Pass creating output_dir:: /mnt/boveda/DOCTORADO2/cnn_emotion4/test_over_video


# Auxiliar functions

In [10]:


def my_func(PredT,frame):
    Clf=PredT[0];
    predictor=PredT[1];

    categories=['negative','neutro','pain','positive'];
    
    img_tmp = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB);
    pil_im = Image.fromarray(img_tmp);
    
    height, width, channels = frame.shape
    
    annotation, gt_anns, image_meta = predictor.pil_image(pil_im);
    
    for annot in annotation: 
        (xi,yi,xo,yo)=oppgd.get_body_bounding_rectangle(annot.data,factor=1.0);
        xi=int(xi);        yi=int(yi);
        xo=int(xo);        yo=int(yo);
        
        if xo<0:
            xo=0;
        if xo>=width:
            xo=width-1;
        if yo<0:
            yo=0;
        if yo>=height:
            yo=height-1;
        
        if xi!=xo and yi!=yo:
            color=(0,255,0);
            thickness=2;

            pil_crop=pil_im.crop((xi,yi,xo,yo));

            res=Clf.from_img_pil(pil_crop);

            texto=categories[res];

            frame = cv2.putText(  frame,
                                  texto,
                                  org = (int(xi), int((yi+yo)/2)),
                                  fontFace = cv2.FONT_HERSHEY_DUPLEX,
                                  fontScale = 2.0,
                                  color = (255, 0, 0),
                                  thickness = thickness
                                )

            cv2.rectangle(frame,(xi,yi),(xo,yo),color,thickness);
    return frame;




In [11]:
    
Clf=mylib.Emotion4Classifier(model_type=model_type,file_of_weight=file_of_weight);
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')#'shufflenetv2k16-wholebody'
vit.apply_func_predictor_over_video_outmp4(my_func,(Clf,predictor),vin_path,vout_path)

print('working end')

Loaded layer with efficientnet_b3


Working: |           [ time left: 00:00 ]      

working end
